In [1]:
import pandas as pd
from pytz import country_timezones
import time
import numpy as np
import timezonefinder
from datetime import datetime as dt
import pytz
import calendar
import seaborn as sns
%matplotlib inline
from matplotlib import rcParams

In [2]:
start_ini = time.time() #check time

In [2]:
import warnings
warnings.filterwarnings('ignore') #remove warning

In [4]:
#import dataset
df = pd.read_csv('https://media.githubusercontent.com/media/DBertazioli/NeoMeetup/master/csv/exported_queries/Distribuzione_temporale_eventi.csv')
df.head()

,ID,time,country,state,lon,lat,partecipanti
0,260269871,5317552800000,us,ca,-118.33,33.87,1
1,260260779,3384554400000,us,wa,-117.43,47.67,2
2,ztbltqyxcdbbc,2150290800000,us,tn,-86.72,36.08,1
3,260361735,1924992000000,sg,sg,103.85,1.30,1
4,260337211,1924992000000,sg,sg,103.85,1.30,1


In [5]:
df['state'] = df['state'].str.lower() #lower charcter 'NONE'-->'none'
df[df['state'] == 'none'].head()

,ID,time,country,state,lon,lat,partecipanti


In [6]:
df.loc[df['state'] == 'none', 'state'] = df['country'] #fill the state's value with country's value if there was 'none'

In [7]:
df.isnull().sum() #check sum of nan fro columns

ID              0
time            0
country         0
state           0
lon             0
lat             0
partecipanti    0
dtype: int64

In [8]:
df = df.dropna() #rm nan value for all df (rows)

In [9]:
df.isnull().any() #check

name       False
time       False
country    False
state      False
lon        False
lat        False
dtype: bool

In [10]:
#df.loc[df['state'] == 'none', 'state'] = df.fillna(value=pd.np.nan, inplace=True) 
#it's only if would take nan into country that do not have a state

In [11]:
#df.isnull().sum() #check

In [12]:
#df.isnull().any() #another check :/

In [8]:
#same method for check nan value into coordinates
#df['lon'] = df['lon'].str.lower() *****crea problemi con NAN su 180k rows di lat******
#df['lat'] = df['lat'].str.lower()
#df.loc[df['lon'] == 'NONE', 'lon'] = df.fillna(value=pd.np.nan, inplace=True)
#df.loc[df['lat'] == 'NONE', 'lat'] = df.fillna(value=pd.np.nan, inplace=True)
#df.isnull().sum()

TypeError: invalid type comparison

In [9]:
df = df.dropna() #remove this nan
df.isnull().sum()

ID              0
time            0
country         0
state           0
lon             0
lat             0
partecipanti    0
dtype: int64

In [10]:
df.tail() #df clean

,ID,time,country,state,lon,lat,partecipanti
280908,tnndrqyzgbcb,1554130800000,fr,fr,2.34,48.86,1
280909,259323127,1554130800000,il,il,34.77,32.07,2
280910,260175682,1554130800000,us,fl,-82.61,28.14,1
280911,260102972,1554130800000,us,az,-111.07,32.09,1
280912,dlqzpqyzgbcb,1554129000000,us,co,-105.08,39.70,1


In [11]:
df['lon'] = df['lon'].astype(float) #convert coordinates into float for problem into cell below (timezonefinder)
df['lat'] = df['lat'].astype(float)
df.dtypes

ID               object
time              int64
country          object
state            object
lon             float64
lat             float64
partecipanti      int64
dtype: object

In [12]:
df = df[[c for c in df if c not in ['lat','lon']] + ['lat','lon']] #change order of column for best position of lat and lng

In [13]:
df = df.reset_index(drop=True) #reset index for resolve eroor of out-bounded

In [14]:
tf = timezonefinder.TimezoneFinder() #defining time zones on lat, long data
df['timezone_str'] = "nan"
count = 0
start = time.time()
for line in df.itertuples():
    count+=1
    try:
        df.timezone_str.at[line.Index] = tf.timezone_at(lat = df.iloc[line.Index]['lat'],
                                                          lng = df.iloc[line.Index]['lon'])
    except Exception as e:
        print(e)
        break
    #if count == 10:
        #break
end = time.time()
print("task completed in "+str((end-start)/60)+" minutes")

task completed in 2.6654537002245586 minutes


In [15]:
df[555:565] #check

,ID,time,country,state,partecipanti,lat,lon,timezone_str
555,ljjfrpybcdblb,1581170400000,us,ga,1,33.94,-83.96,America/New_York
556,njrfgpybcdblb,1581166800000,us,ga,1,33.94,-83.96,America/New_York
557,jktqnpybcdbkb,1581062400000,au,au,1,-27.46,153.02,Australia/Brisbane
558,260411081,1581044400000,us,ca,2,38.58,-121.51,America/Los_Angeles
559,260468041,1581039000000,us,co,1,39.70,-105.08,America/Denver
560,260432130,1580995800000,gb,gb,7,53.81,-1.55,Europe/London
561,xbcxkmybcdbjb,1580992200000,us,fl,2,28.66,-81.37,America/New_York
562,hwnrdpybcdbhb,1580956200000,us,ca,1,37.07,-122.01,America/Los_Angeles
563,260478071,1580948100000,us,nc,3,35.77,-78.64,America/New_York
564,cmwcmqybcdbhb,1580933700000,gb,gb,1,55.94,-3.22,Europe/London


In [16]:
df.dtypes

ID               object
time              int64
country          object
state            object
partecipanti      int64
lat             float64
lon             float64
timezone_str     object
dtype: object

In [22]:
#this step is dated (only for convert apoc in neo)
#df['time'] = pd.to_datetime(df['time']) #convert time (object) into datettime type (needed fro convert after into timestamp)

In [23]:
##this step is dated (only for convert apoc in neo)
'''df['timestamp'] = df.fillna(value=pd.np.nan, inplace=True) #fill with nan value
start = time.time()
count = 0
for line in df.itertuples():
    count+=1
    try:
        df['timestamp'].at[line.Index] = dt.timestamp(df.iloc[line.Index]['time'])
    except Exception as e:
        print(e)
        break
    #if count == 100:
        #break
end = time.time()
print('task completed in '+str((end-start)/60)+' minutes')'''

"df['timestamp'] = df.fillna(value=pd.np.nan, inplace=True) #fill with nan value\nstart = time.time()\ncount = 0\nfor line in df.itertuples():\n    count+=1\n    try:\n        df['timestamp'].at[line.Index] = dt.timestamp(df.iloc[line.Index]['time'])\n    except Exception as e:\n        print(e)\n        break\n    #if count == 100:\n        #break\nend = time.time()\nprint('task completed in '+str((end-start)/60)+' minutes')"

In [17]:
df.head(10) #check

,ID,time,country,state,partecipanti,lat,lon,timezone_str
0,260269871,5317552800000,us,ca,1,33.87,-118.33,America/Los_Angeles
1,260260779,3384554400000,us,wa,2,47.67,-117.43,America/Los_Angeles
2,ztbltqyxcdbbc,2150290800000,us,tn,1,36.08,-86.72,America/Chicago
3,260361735,1924992000000,sg,sg,1,1.30,103.85,Asia/Singapore
4,260337211,1924992000000,sg,sg,1,1.30,103.85,Asia/Singapore
5,260361770,1924992000000,sg,sg,1,1.30,103.85,Asia/Singapore
6,260276424,1924992000000,sg,sg,1,1.30,103.85,Asia/Singapore
7,260337185,1924992000000,sg,sg,1,1.30,103.85,Asia/Singapore
8,260361712,1924992000000,sg,sg,1,1.30,103.85,Asia/Singapore
9,260532116,1924992000000,sg,sg,1,1.30,103.85,Asia/Singapore


In [18]:
def rounder(t): #this function round the time to nearest hour
    if t.minute >= 30:
        if t.hour == 23:
            return t.replace(second=0, microsecond=0, minute=0, hour=0)
        else:
            return t.replace(second=0, microsecond=0, minute=0, hour= t.hour+1)
    else:
        return t.replace(second=0, microsecond=0, minute=0)

In [19]:
df['datetime_timezoned'] = df.fillna(value=pd.np.nan, inplace=True) #fill with nan value

In [20]:
df['datetime_timezoned'] = df['datetime_timezoned'].astype(object) #convert for resolve error in the cell above (should not be a number)
df.dtypes

ID                     object
time                    int64
country                object
state                  object
partecipanti            int64
lat                   float64
lon                   float64
timezone_str           object
datetime_timezoned     object
dtype: object

In [21]:
start = time.time()
count = 0
for line in df.itertuples():
    count+=1
    try:
        df.datetime_timezoned.at[line.Index] = rounder((dt.fromtimestamp
                                                   (timestamp = (df.iloc[line.Index]['time'])/1000,
                                                    tz = pytz.timezone(df.iloc[line.Index]['timezone_str'])
                                                   ))).strftime("%d/%m/%Y %H")
    except Exception as e:
        print(e)
        break
    #if count == 100:
        #break
end = time.time()
print('task completed in '+str((end-start)/60)+' minutes')

task completed in 2.6073269406954447 minutes


In [23]:
df = df[[c for c in df if c not in ['timezone_str','time','datetime_timezoned']] + ['timezone_str','time','datetime_timezoned']]
df.tail(10) #re-order column

,ID,country,state,partecipanti,lat,lon,timezone_str,time,datetime_timezoned
280903,260208676,us,nc,1,35.92,-79.12,America/New_York,1554130800000,01/04/2019 11
280904,260214529,ae,ae,1,25.27,55.33,Asia/Dubai,1554130800000,01/04/2019 19
280905,259955131,us,il,1,41.81,-88.02,America/Chicago,1554130800000,01/04/2019 10
280906,260247643,us,ut,1,40.87,-111.89,America/Denver,1554130800000,01/04/2019 09
280907,260049652,de,de,1,52.52,13.38,Europe/Berlin,1554130800000,01/04/2019 17
280908,tnndrqyzgbcb,fr,fr,1,48.86,2.34,Europe/Paris,1554130800000,01/04/2019 17
280909,259323127,il,il,2,32.07,34.77,Asia/Jerusalem,1554130800000,01/04/2019 18
280910,260175682,us,fl,1,28.14,-82.61,America/New_York,1554130800000,01/04/2019 11
280911,260102972,us,az,1,32.09,-111.07,America/Phoenix,1554130800000,01/04/2019 08
280912,dlqzpqyzgbcb,us,co,1,39.70,-105.08,America/Denver,1554129000000,01/04/2019 09


In [29]:
#df = df[[c for c in df if c not in ['timezone_str','time','datetime_timezoned']] + ['timezone_str','time','datetime_timezoned']]
#df.head(10) #re-order column

,name,country,state,lat,lon,timezone_str,time,datetime_timezoned
0,Lunch meetup at Shanghai Slims 13th April Sat...,cn,cn,31.23,121.47,Asia/Shanghai,1.555128e+12,13/04/2019 12:00:00
1,"\t Meetup Salud M\u00f3vil, IA Salud - Observa...",co,co,6.25,-75.59,America/Bogota,1.554413e+12,04/04/2019 17:00:00
2,\t Rob Thomas: Chip Tooth Tour,us,co,39.70,-105.08,America/Denver,1.560213e+12,10/06/2019 19:00:00
3,\tHayden Island,us,or,45.44,-122.77,America/Los_Angeles,1.554568e+12,06/04/2019 10:00:00
4,\tNorth - University of Portland,us,or,45.44,-122.77,America/Los_Angeles,1.555173e+12,13/04/2019 10:00:00
5,\tPhunsukh Wangdu Isolated Camping,in,in,12.97,77.56,Asia/Kolkata,1.554532e+12,06/04/2019 12:00:00
6,"\tSevern Shires Way part 4, Farnborough to Mid...",gb,gb,51.76,-1.26,Europe/London,1.554628e+12,07/04/2019 10:00:00
7,\tSpeak EZ Toastemasters,us,tx,32.96,-96.75,America/Chicago,1.555024e+12,11/04/2019 18:00:00
8,\tSpeak EZ Toastemasters,us,tx,32.96,-96.75,America/Chicago,1.554419e+12,04/04/2019 18:00:00
9,\tWine and Paint Party - Unicorn Painting,nz,nz,-36.85,174.76,Pacific/Auckland,1.555206e+12,14/04/2019 14:00:00


In [24]:
df.to_csv('../csv/exported_queries/Distribuzione_temporale_eventi_fix_with_members_replica.csv') #csv

In [31]:
end_fin = time.time()
print("script completed in "+str((end_fin-start_ini)/60)+" minutes")

script completed in 3.812490542729696 minutes
